In [26]:
import h5py
positions = h5py.File("data/Data.h5", 'r')
keys = positions.keys()
for key in keys:
    print(key)
    print(positions[key])


VID_RGB_000.csv
<HDF5 dataset "VID_RGB_000.csv": shape (30, 48), type "<f8">
VID_RGB_001.csv
<HDF5 dataset "VID_RGB_001.csv": shape (30, 48), type "<f8">
VID_RGB_002.csv
<HDF5 dataset "VID_RGB_002.csv": shape (30, 48), type "<f8">
VID_RGB_003.csv
<HDF5 dataset "VID_RGB_003.csv": shape (30, 48), type "<f8">
VID_RGB_004.csv
<HDF5 dataset "VID_RGB_004.csv": shape (30, 48), type "<f8">
VID_RGB_005.csv
<HDF5 dataset "VID_RGB_005.csv": shape (30, 48), type "<f8">
VID_RGB_006.csv
<HDF5 dataset "VID_RGB_006.csv": shape (30, 48), type "<f8">
VID_RGB_007.csv
<HDF5 dataset "VID_RGB_007.csv": shape (30, 48), type "<f8">
VID_RGB_008.csv
<HDF5 dataset "VID_RGB_008.csv": shape (30, 48), type "<f8">
VID_RGB_009.csv
<HDF5 dataset "VID_RGB_009.csv": shape (30, 48), type "<f8">
VID_RGB_010.csv
<HDF5 dataset "VID_RGB_010.csv": shape (30, 48), type "<f8">
VID_RGB_011.csv
<HDF5 dataset "VID_RGB_011.csv": shape (30, 48), type "<f8">
VID_RGB_012.csv
<HDF5 dataset "VID_RGB_012.csv": shape (30, 48), type "<f8">

In [49]:
import h5py
import numpy as np

file_path = "data/Data.h5"

dataset_arrays = []  # List chứa từng dataset dạng numpy array (30, 48)

with h5py.File(file_path, "r") as f:
    def process_dataset(name, obj):
        if isinstance(obj, h5py.Dataset):
            data = obj[()]
            # Kiểm tra kích thước để đảm bảo (30,48)
            if data.shape == (30, 48):
                dataset_arrays.append(data)
            else:
                print(f"⚠ Dataset '{name}' có kích thước {data.shape}, bỏ qua.")

    f.visititems(process_dataset)

# Chuyển list thành numpy array (shape: [số dataset, 30, 48])
Gait = np.array(dataset_arrays)

print(f"Số dataset đã lấy: {Gait.shape[0]}")
print(f"Kích thước mảng cuối cùng: {Gait.shape}")


Số dataset đã lấy: 672
Kích thước mảng cuối cùng: (672, 30, 48)


In [50]:
import h5py
import pandas as pd
import numpy as np

file_path = "data/Labels_My.h5"  # chỉnh đúng đường dẫn

row_1_values_all = []  # mảng chứa tất cả giá trị hàng 1 các dataset

with h5py.File(file_path, "r") as f:
    def process_dataset(name, obj):
        if isinstance(obj, h5py.Dataset):
            # Đọc toàn bộ dữ liệu dataset
            data = obj[()]
            df = pd.DataFrame(data)

            # Kiểm tra dataset có ít nhất 2 hàng không để lấy hàng 1
            if df.shape[0] > 1:
                # Lấy giá trị hàng 1 (index=1) dạng numpy array
                row_1 = df.iloc[1].values

                # Thêm từng phần tử trong hàng 1 vào mảng chung
                row_1_values_all.extend(row_1.tolist())
            else:
                print(f"⚠ Dataset {name} không có đủ 2 hàng để lấy hàng 1.")

    f.visititems(process_dataset)

# Chuyển list thành numpy array (tùy chọn)
labels = np.array(row_1_values_all)

print("Tổng giá trị hàng 1 của tất cả dataset gộp lại:")
print(labels.shape)


Tổng giá trị hàng 1 của tất cả dataset gộp lại:
(672,)


In [55]:
import torch.nn as nn

class VanillaLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(VanillaLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out


In [53]:
from sklearn.model_selection import train_test_split

X = Gait

# Ví dụ tạo label giả, vì bạn chưa cung cấp label (thường cần cho supervised learning)
# Đây chỉ là ví dụ: giả sử tất cả label là 0
y = labels

# Chia train-test (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True)


In [68]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 4. Chuyển dữ liệu sang tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Tạo DataLoader
batch_size = 16
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [69]:
input_size = 48
hidden_size = 64
output_size = 4
num_layers = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = VanillaLSTM(input_size, hidden_size, output_size, num_layers).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Using device: cuda


In [70]:
num_epochs = 500

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)               # (batch_size, 4)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


Epoch 1/500, Loss: 1.3172
Epoch 2/500, Loss: 1.2652
Epoch 3/500, Loss: 1.2587
Epoch 4/500, Loss: 1.2530
Epoch 5/500, Loss: 1.2364
Epoch 6/500, Loss: 1.2622
Epoch 7/500, Loss: 1.2351
Epoch 8/500, Loss: 1.2232
Epoch 9/500, Loss: 1.1939
Epoch 10/500, Loss: 1.1991
Epoch 11/500, Loss: 1.1998
Epoch 12/500, Loss: 1.1874
Epoch 13/500, Loss: 1.1802
Epoch 14/500, Loss: 1.1937
Epoch 15/500, Loss: 1.1642
Epoch 16/500, Loss: 1.1432
Epoch 17/500, Loss: 1.1392
Epoch 18/500, Loss: 1.1083
Epoch 19/500, Loss: 1.0944
Epoch 20/500, Loss: 1.1338
Epoch 21/500, Loss: 1.0735
Epoch 22/500, Loss: 1.0959
Epoch 23/500, Loss: 1.0544
Epoch 24/500, Loss: 1.1375
Epoch 25/500, Loss: 1.0870
Epoch 26/500, Loss: 1.0767
Epoch 27/500, Loss: 1.0901
Epoch 28/500, Loss: 1.0240
Epoch 29/500, Loss: 1.0407
Epoch 30/500, Loss: 1.0043
Epoch 31/500, Loss: 0.9853
Epoch 32/500, Loss: 1.0223
Epoch 33/500, Loss: 1.0046
Epoch 34/500, Loss: 0.9837
Epoch 35/500, Loss: 1.0060
Epoch 36/500, Loss: 0.8973
Epoch 37/500, Loss: 0.9359
Epoch 38/5

In [71]:
model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    outputs = model(X_test_tensor)
    predicted = torch.argmax(outputs, dim=1)
    accuracy = (predicted == y_test_tensor).float().mean()
    print(f"Test accuracy: {accuracy:.4f}")

Test accuracy: 0.9185


In [72]:
torch.save(model, "lstm_model_full.pth")

In [74]:
model = torch.load("lstm_model_full.pth", weights_only=False)
lstm_layer = model.lstm